In [76]:
import pandas as pd 
import numpy as np
import unidecode
import spacy
import re

In [ ]:
df = pd.read_csv("ExpedientesPublicados.csv")

In [ ]:
nlp = spacy.load("es_core_news_md")

In [86]:
def process_text(text):
    text = ' '.join(text.split())
    doc = nlp(text.lower().replace("¨",""))
    result = []
    for token in doc:
        if '"' in token.text:
            continue
        if '-' in token.text:
            continue
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    #Remove accents
    result = unidecode.unidecode(" ".join(result))
    result = re.sub(r"[^A-Za-z\s]+", ' ', result)
    return result

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95353 entries, 0 to 96121
Data columns (total 22 columns):
Código del expediente           95353 non-null int64
Número del procedimiento        95353 non-null object
Carácter del procedimiento      95353 non-null object
Forma del procedimiento         95353 non-null object
Artículo de excepción           95353 non-null object
REFERENCIA_EXPEDIENTE           95353 non-null object
Título del expediente           95353 non-null object
Plantilla del expediente        95353 non-null object
Descripción del Anuncio         95353 non-null object
Clave de la UC                  95353 non-null object
Nombre de la UC                 95353 non-null object
Operador                        95353 non-null object
Correo electrónico              95353 non-null object
Entidad federativa              95353 non-null object
Tipo de contratación            95353 non-null object
Publicación del anuncio         95353 non-null object
Vigencia del anuncio        

In [87]:
print(process_text('SERVICIOS PROFESIONALES PARA COADYUVAREN EL REGISTRO Y CONTROL DE LOS PROYECTOSDE INVESTIGACIÓN, EN LA CREACIÓN DEEXPEDIENTES ACADÉMICOS Y FÍSICOS EN ELSISTEMA ALBALÁ, APOYAR EN LA REALIZACIÓNDE LOS TRÁMITES REFERENTES A LACOORDINACIÓN DE INVESTIGACIÓN DELCENTRO NACIONAL DE INVESTIGACIÓN,DOCUMENTACIÓN E INVESTIGACIÓN MUSICAL“CARLOS CHÁVEZ”.'))

servicio profesional coadyuvar registrar y control proyectosde investigacion creacion deexpedientes academico y fisico elsistema albala apoyar realizacionde tramite referente a lacoordinacion investigacion delcentro nacional investigacion documentacion e investigacion musical carlos chavez


In [88]:
df = df.fillna('')
df['CleanDesc'] = df.apply(lambda row : process_text(row['Descripción del Anuncio']), axis = 1)
descriptions = df['CleanDesc'].value_counts()
descriptions.head(60)

servicio profesional elaboracion avaluo                                                                                                                                                                                                                                                                                         1915
harina maiz                                                                                                                                                                                                                                                                                                                     1226
servicio profesional                                                                                                                                                                                                                                                                                                             915
material curacion        

In [89]:
df = df[df['CleanDesc'].astype(bool)]

In [90]:
df['CleanDesc'].sample(20)

28040              servicio profesional elaboracion avaluo
77960                              adquisicion medicamento
73895               adquisicion consumible equipar medicar
80523                                      papel higienico
31817    servicio apoyar a direccion recurso material y...
55366    trabajo obrar civil e instalacion electromecan...
48641                     servicio telefonico convencional
61657                                          grupo      
20101                                        jugo y nectar
63062    elaboracion estudio socavacion y evaluacion es...
17756           rehabilitacion primario guadalupe victoria
31923    servicio preservacion digital a soporte iconog...
53680                         adq material curacion covid 
19789    servicio limpieza a instalacion aeropuerto int...
1982               servicio profesional elaboracion avaluo
10412                     adquisicion colchon hopitalarios
75985                              material complementar

In [96]:
for row in df['CleanDesc']:
    if '"' in row:
        print(row)

In [91]:
df['Word Vectors'] = df.apply(lambda row : nlp(row['CleanDesc']).vector, axis = 1)

In [92]:
df.to_csv('ExpedientesNLP.csv')

PermissionError: [Errno 13] Permission denied: 'ExpedientesNLP.csv'

In [97]:
clustering_df = df[['CleanDesc','Word Vectors']]
vector = pd.DataFrame(clustering_df['Word Vectors'].tolist(), index= clustering_df.index)
clustering_df = clustering_df.merge(vector, left_index=True, right_index=True)
clustering_df = clustering_df.drop(['Word Vectors'], axis = 1)
clustering_df.head()

,CleanDesc,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,mejoramiento iluminacion iglesia santo miguel,0.285134,-0.347190,0.372838,1.786500,1.304575,-0.875542,0.241736,2.207744,1.386242,...,0.015060,0.083816,-0.624450,1.240238,1.526300,-0.409667,1.292290,-0.202832,-0.500314,-0.494364
1,adquisicion uniforme direccion seguridad publi...,1.505920,-0.214450,0.031016,-0.216319,-0.079253,-0.846233,-0.163367,1.464723,0.367282,...,-0.208350,0.192616,-0.442881,0.968356,0.985995,-0.190079,1.071147,0.525607,-1.122517,0.314290
2,proyectar ejecutivo jardin vecinal colonia ame...,0.879200,-0.109962,1.224388,0.414138,1.023179,0.381762,-0.782174,0.455059,0.482879,...,0.245520,0.150462,0.823416,0.994631,1.179548,-0.024129,1.741747,0.117264,1.272586,0.618184
3,construccion deportivo bicentenario,1.647743,-0.124789,0.974133,0.514520,1.243167,-0.244903,-0.728567,0.235247,0.193003,...,-0.948187,0.531850,-1.003360,0.536137,1.164490,-1.412653,1.416800,0.748527,1.055030,-1.647617
4,acondicionamiento centrar cultural y deportivo,1.590622,0.248824,1.003108,-0.054634,1.140828,-0.664088,-0.539960,0.613790,0.202512,...,-0.214696,0.146978,-0.556054,-0.291617,1.160667,-1.053317,1.052188,-0.020922,0.222214,-1.146474


In [98]:
clustering_df.to_csv('DescVectors.csv',index=False)

In [14]:
suma = 0
for i,element in enumerate(descriptions):
    suma += element
    if suma > 96122/2:
        print(suma)
        print(i)
        print(descriptions.index[i])
        break


48062
9256
investigacion mercar servicio mantenimiento preventivo y correctivo a equipo distribucion y suministrar electrico generador vapor y compresor aire presion hidroneumatico y bombear aire acondicionar y sistema proteccion incendio instalar inmueble ccayac y eq minisplits instalar inmueble oklahoma constar 6 partir 1 mantto preventivo y correctivo a subestacion electrico tablero electrico distribucion subgeneral y centro cargar transformador electrico bajo potenciar plantar electrico emergencia sistema tierra pararrayos unidad ininterrumpibles energia ups 2 mantto preventivo y correctivo a centro control motor generador vapor caldera caldereta compresor aire presion positivo y negativo cabina banar aire air shower pass box tranfer's electronico 3 mantto preventivo y correctivo a sistema hidroneumatico eq bombear carcamo eq recirculadores aguar y bombear aguar potable sistema almacenamiento y distribucion gas licuar petroleo tanque almacenamiento regulador presion manometro valvu

In [19]:
main_options = np.where(descriptions>10,descriptions,'')
main_options = main_options[main_options != '']

In [20]:
print(len(main_options))

401


In [73]:
doc = nlp("maíz maiz plátano platano tortilla")
for token in doc:
    for token2 in doc:
        print("Sim: " + token + " , " + token2)
        print(token.similarity(token1))

 
SPACE
False
hello
PROPN
False
-
PUNCT
True
my
PRON
False
'
PUNCT
True
d
INTJ
False
fiend
PROPN
False
